In [14]:
#Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
# Importing Dataset
data=pd.read_csv('Data/Flight_Data.csv',nrows=1000)


NameError: name 'pd' is not defined

In [21]:
data.shape

(1000, 27)

In [19]:
data.columns

Index(['legId', 'searchDate', 'flightDate', 'startingAirport',
       'destinationAirport', 'fareBasisCode', 'travelDuration', 'elapsedDays',
       'isBasicEconomy', 'isRefundable', 'isNonStop', 'baseFare', 'totalFare',
       'seatsRemaining', 'totalTravelDistance',
       'segmentsDepartureTimeEpochSeconds', 'segmentsDepartureTimeRaw',
       'segmentsArrivalTimeEpochSeconds', 'segmentsArrivalTimeRaw',
       'segmentsArrivalAirportCode', 'segmentsDepartureAirportCode',
       'segmentsAirlineName', 'segmentsAirlineCode',
       'segmentsEquipmentDescription', 'segmentsDurationInSeconds',
       'segmentsDistance', 'segmentsCabinCode'],
      dtype='object')

In [26]:
data.iloc[456]

legId                                                 df29acaa40fca38c7a9905e6249df687
searchDate                                                                  2022-06-13
flightDate                                                                  2022-06-15
startingAirport                                                                    ATL
destinationAirport                                                                 EWR
fareBasisCode                                                                      YNR
travelDuration                                                                 PT8H53M
elapsedDays                                                                          0
isBasicEconomy                                                                   False
isRefundable                                                                     False
isNonStop                                                                        False
baseFare                                   

In [18]:
data[['flightDate','startingAirport','fareBasisCode']].tail(n=2)

,flightDate,startingAirport,fareBasisCode
998,2022-06-03,JFK,M7AHZNN3
999,2022-06-12,BOS,SAA0OHEN
